In [ ]:
# default_exp data.core

In [ ]:
#hide
%reload_ext autoreload
%autoreload 2
%matplotlib inline

# data.core

> This module contains the core bits required to use the fastai DataBlock API and/or mid-level data processing pipelines to organize your data in a way modelable by huggingface transformer implementations.

In [ ]:
#export
from functools import reduce

import torch
from transformers import *
from fastai2.text.all import *

from blurr.utils import *

In [ ]:
#hide
import pdb

from nbdev.showdoc import *
from fastcore.test import *

In [ ]:
#cuda
torch.cuda.set_device(1)
print(f'Using GPU #{torch.cuda.current_device()}: {torch.cuda.get_device_name()}')

Using GPU #1: GeForce GTX 1080 Ti


## Base tokenization, batch transform, and DataBlock methods

In [ ]:
#export
class HF_BaseInput(list): pass

The `HF_BaseInput` object is used to encapsulate all the inputs required by whatever huggingface model we are using. We use it as a container for the `input_ids`, `token_type_ids`, and `attention_mask` tensors required by most models, and also as a mean to customize @typedispatched functions like `DataLoaders.show_batch` and `Learner.show_results`.

In [ ]:
#export
class HF_TokenizerTransform(Transform):
    """huggingface friendly tokenization transfor."""
    def __init__(self, hf_arch, hf_tokenizer, title='text', **kwargs):
        
        store_attr(self, 'hf_arch, hf_tokenizer, title')
        self.add_prefix_space = hf_arch in ['gpt2', 'roberta', 'bart']
        
    def encodes(self, inp): 
        """Supports both string and list[str] inputs (the later is common for token classification tasks).
        Returns the numericalized (token_ids) of the input so no need to run this through a Numericalization
        transform."""
        if (isinstance(inp, str)): 
            toks = self.hf_tokenizer.tokenize(inp, add_prefix_space=self.add_prefix_space)
        else:
            toks = [sub_toks for entity in inp 
                    for sub_toks in self.hf_tokenizer.tokenize(entity, add_prefix_space=self.add_prefix_space)]
            
        return tensor(self.hf_tokenizer.convert_tokens_to_ids(toks))
    
    def decodes(self, encoded_inp): 
        """This will get called multiple times for a given encoded input because our batch transform will add
        other elements to it (e.g., attention_mask, token_type_ids, etc...) as required by the defined huggingface
        tokenizer and model.  If it can't decode it, return None."""
        try: return TitledStr(self.hf_tokenizer.decode(encoded_inp.cpu().numpy()))
        except: return None
    

`HF_TokenizerTransform` was inspired by [this article](http://dev.fast.ai/tutorial.transformers).  It handles both the tokenization and numericalization traditionally split apart in the fastai text DataBlock API.  Addtionally, it's been updated to add a prefix space for the huggingface architectures that need it.

You can pass a string or list into this Transform, the later being common in token classification tasks like named entity recognition.

In [ ]:
#export
@typedispatch
def build_hf_input(task, tokenizer, a_tok_ids, b_tok_ids=None, targets=None,
                   max_length=512, pad_to_max_length=True, truncation_strategy='longest_first'):

    res = tokenizer.prepare_for_model(a_tok_ids, b_tok_ids, 
                                       max_length=max_length, pad_to_max_length=pad_to_max_length,
                                       truncation_strategy=truncation_strategy, return_tensors='pt')
    
    input_ids = res['input_ids'][0]
    attention_mask = res['attention_mask'][0] if ('attention_mask' in res) else torch.tensor([-9999]) 
    token_type_ids = res['token_type_ids'][0] if ('token_type_ids' in res) else torch.tensor([-9999]) 
    
    return HF_BaseInput([input_ids, attention_mask, token_type_ids]), targets

`build_hf_input` uses fastai's @typedispatched decorator to provide for complete flexibility in terms of how your numericalized tokens are assembled, and also what you return via `HF_BaseInput` and as your targets.  You can override this implementation as needed by assigning a type to the `task` argument (and optionally the `tokenizer` argument as well).

What you return here is what will be fed into your huggingface model.

In [ ]:
#export
class HF_BatchTransform(Transform):
    """Handles everything you need to assemble a mini-batch of inputs and targets"""
    def __init__(self, hf_arch, hf_tokenizer, max_seq_len=512, truncation_strategy='longest_first', task=None, 
                 **kwargs):
        
        self.hf_arch = hf_arch
        self.hf_tokenizer = hf_tokenizer
        store_attr(self, 'max_seq_len, truncation_strategy, task, kwargs')
        
    def encodes(self, samples):
        encoded_samples = []
        for idx, sample in enumerate(samples):
            if (isinstance(sample[0], tuple)):
                a_tok_ids, b_tok_ids = sample[0][0].tolist(), sample[0][1].tolist()
            else:
                a_tok_ids, b_tok_ids = sample[0].tolist(), None

            hf_base_input, targets = build_hf_input(self.task, self.hf_tokenizer, 
                                                    a_tok_ids, b_tok_ids, sample[1:],
                                                    self.max_seq_len, True, self.truncation_strategy, **self.kwargs)
            
            encoded_samples.append((hf_base_input, *targets))
            
        return encoded_samples

In [ ]:
isinstance(None, ForConditionalGenerationTask)

False

In [ ]:
#export
class HF_TextBlock(TransformBlock):
    def __init__(self, hf_arch, hf_tokenizer, hf_batch_tfm=None, max_seq_len=512, task=None, **kwargs):
        
        if (hf_batch_tfm is None): 
            hf_batch_tfm = HF_BatchTransform(hf_arch, hf_tokenizer, max_seq_len, task=task, **kwargs)
            
        dl_type = SortedDL
        if (isinstance(task, ForConditionalGenerationTask)): dl_type=None
    
        return super().__init__(type_tfms=HF_TokenizerTransform(hf_arch, hf_tokenizer), 
                                dl_type=dl_type, dls_kwargs={ 'before_batch': hf_batch_tfm })            

`HF_TextBlock` has been dramatically simplified from it's predecessor.  It handles setting up your `HF_TokenizerTransform` transforms and `HF_BatchTransform` transform regardless of data source (e.g., this will work with files, DataFrames, whatever).

In [ ]:
#export
@typedispatch
def show_batch(x:HF_BaseInput, y, samples, hf_tokenizer, skip_special_tokens=True, ctxs=None, max_n=6, **kwargs):  
    if ctxs is None: ctxs = get_empty_df(min(len(samples), max_n))
        
    samples = L((TitledStr(hf_tokenizer.decode(inp, skip_special_tokens=skip_special_tokens).replace(hf_tokenizer.pad_token, '')), *s[1:]) 
                for inp, s in zip(x[0], samples))
    
    ctxs = show_batch[object](x, y, samples, max_n=max_n, ctxs=ctxs, **kwargs)

    display_df(pd.DataFrame(ctxs))
    return ctxs

## Example usage

Below demonstrates how to contruct your `DataBlock` for a sequence classification task (e.g., a model that requires a single text input)

In [ ]:
path = untar_data(URLs.IMDB_SAMPLE)

model_path = Path('models')
imdb_df = pd.read_csv(path/'texts.csv')

In [ ]:
imdb_df.head()

,label,text,is_valid
0,negative,"Un-bleeping-believable! Meg Ryan doesn't even look her usual pert lovable self in this, which normally makes me forgive her shallow ticky acting schtick. Hard to believe she was the producer on this dog. Plus Kevin Kline: what kind of suicide trip has his career been on? Whoosh... Banzai!!! Finally this was directed by the guy who did Big Chill? Must be a replay of Jonestown - hollywood style. Wooofff!",False
1,positive,"This is a extremely well-made film. The acting, script and camera-work are all first-rate. The music is good, too, though it is mostly early in the film, when things are still relatively cheery. There are no really superstars in the cast, though several faces will be familiar. The entire cast does an excellent job with the script.<br /><br />But it is hard to watch, because there is no good end to a situation like the one presented. It is now fashionable to blame the British for setting Hindus and Muslims against each other, and then cruelly separating them into two countries. There is som...",False
2,negative,"Every once in a long while a movie will come along that will be so awful that I feel compelled to warn people. If I labor all my days and I can save but one soul from watching this movie, how great will be my joy.<br /><br />Where to begin my discussion of pain. For starters, there was a musical montage every five minutes. There was no character development. Every character was a stereotype. We had swearing guy, fat guy who eats donuts, goofy foreign guy, etc. The script felt as if it were being written as the movie was being shot. The production value was so incredibly low that it felt li...",False
3,positive,"Name just says it all. I watched this movie with my dad when it came out and having served in Korea he had great admiration for the man. The disappointing thing about this film is that it only concentrate on a short period of the man's life - interestingly enough the man's entire life would have made such an epic bio-pic that it is staggering to imagine the cost for production.<br /><br />Some posters elude to the flawed characteristics about the man, which are cheap shots. The theme of the movie ""Duty, Honor, Country"" are not just mere words blathered from the lips of a high-brassed offic...",False
4,negative,"This movie succeeds at being one of the most unique movies you've seen. However this comes from the fact that you can't make heads or tails of this mess. It almost seems as a series of challenges set up to determine whether or not you are willing to walk out of the movie and give up the money you just paid. If you don't want to feel slighted you'll sit through this horrible film and develop a real sense of pity for the actors involved, they've all seen better days, but then you realize they actually got paid quite a bit of money to do this and you'll lose pity for them just like you've alr...",False


There are a bunch of ways we can get at the four huggingface elements we need (e.g., architecture name, tokenizer, config, and model).  We can just create them directly, or we can use one of the helper methods available via `BLURR_MODEL_HELPER`.

In [ ]:
task = HF_TASKS_AUTO.ForSequenceClassification

pretrained_model_name = "roberta-base" # "distilbert-base-uncased" "bert-base-uncased"
config = AutoConfig.from_pretrained(pretrained_model_name)

hf_arch, hf_tokenizer, hf_config, hf_model = BLURR_MODEL_HELPER.get_auto_hf_objects(pretrained_model_name, 
                                                                                    task=task, 
                                                                                    config=config)

Once you have those elements, you can create your `DataBlock` as simple as the below.

In [ ]:
# single input
blocks = (HF_TextBlock(hf_arch=hf_arch, hf_tokenizer=hf_tokenizer), CategoryBlock)

dblock = DataBlock(blocks=blocks, 
                   get_x=ColReader('text'), get_y=ColReader('label'), 
                   splitter=ColSplitter(col='is_valid'))

In [ ]:
# dblock.summary(imdb_df)

In [ ]:
dls = dblock.dataloaders(imdb_df, bs=4)

In [ ]:
b = dls.one_batch(); len(b), len(b[0]), len(b[1]) 

(2, 3, 4)

In [ ]:
b[0][0].shape, b[0][1].shape, b[0][2].shape, b[1].shape

(torch.Size([4, 512]),
 torch.Size([4, 512]),
 torch.Size([4, 1]),
 torch.Size([4]))

In [ ]:
dls.show_batch(hf_tokenizer=hf_tokenizer, max_n=2)

,text,category
0,"Raising Victor Vargas: A Review<br /><br />You know, Raising Victor Vargas is like sticking your hands into a big, steaming bowl of oatmeal. It's warm and gooey, but you're not sure if it feels right. Try as I might, no matter how warm and gooey Raising Victor Vargas became I was always aware that something didn't quite feel right. Victor Vargas suffers from a certain overconfidence on the director's part. Apparently, the director thought that the ethnic backdrop of a Latino family on the lower east side, and an idyllic storyline would make the film critic proof. He was right, but it didn't fool me. Raising Victor Vargas is the story about a seventeen-year old boy called, you guessed it, Victor Vargas (Victor Rasuk) who lives his teenage years chasing more skirt than the Rolling Stones could do in all the years they've toured. The movie starts off in `Ugly Fat' Donna's bedroom where Victor is sure to seduce her, but a cry from outside disrupts his plans when his best-friend Harold (Kevin Rivera) comes-a-looking for him. Caught in the attempt by Harold and his sister, Victor Vargas runs off for damage control. Yet even with the embarrassing implication that he's been boffing the homeliest girl in the neighborhood, nothing dissuades young Victor from going off on the hunt for more fresh meat. On a hot, New York City day they make way to the local public swimming pool where Victor's eyes catch a glimpse of the lovely young nymph Judy (Judy Marte), who's not just pretty, but a strong and independent too. The relationship that develops between Victor and Judy becomes the focus of the film. The story also focuses on Victor's family that is comprised of his grandmother or abuelita (Altagracia Guzman), his brother Nino (also played by real life brother to Victor, Silvestre Rasuk) and his sister Vicky (Krystal Rodriguez). The action follows Victor between scenes with Judy and scenes with his family. Victor tries to cope with being an oversexed pimp-daddy, his feelings for Judy and his grandmother's conservative Catholic upbringing.<br /><br />The problems that arise from Raising Victor Vargas are a few, but glaring errors. Throughout the film you get to know certain characters like Vicky, Nino, Grandma,",negative
1,"Well, what can I say.<br /><br />""What the Bleep do we Know"" has achieved the nearly impossible - leaving behind such masterpieces of the genre as ""The Postman"", ""The Dungeon Master"", ""Merlin"", and so fourth, it will go down in history as the single worst movie I have ever seen in its entirety. And that, ladies and gentlemen, is impressive indeed, for I have seen many a bad movie.<br /><br />This masterpiece of modern cinema consists of two interwoven parts, alternating between a silly and contrived plot about an extremely annoying photographer, abandoned by her husband and forced to take anti-depressants to survive, and a bunch of talking heads going on about how quantum physics supposedly justifies their new-agy pseudo-philosophy. Basically, if you start your day off meditating to the likes of Enya and Kenny G, this movie is for you. If you have a sense of humor, a crowd of people who know how to have fun, and a sizable portion of good weed, then this movie is for you as well. Otherwise, stay away. Take my word for it.<br /><br />The first thing that struck me about ""What the Bleep do you Know"" is that is seemed to be edited and put together by the same kinds of people that shoot cheap weddings on camera, complete with pink heart effects, computer-generated sparkles across the screen, and other assorted silliness. Who let these people anywhere near a theatrical release is a mystery to me. I guess this is what too much Kenny G does to you. The movie was permeated with cheesy GCI, the likes that you or I can produce on our own computer via over-the-counter video editing software, but never would, because it's just way too ridiculous.<br /><br />The script was _obviously_ written by someone with n

## Cleanup

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_utils.ipynb.
Converted 01_data-core.ipynb.
Converted 01a_data-language-modeling.ipynb.
Converted 01c_data-question-answering.ipynb.
Converted 01d_data-token-classification.ipynb.
Converted 01e_data-text-generation.ipynb.
Converted 02_modeling-core.ipynb.
Converted 02a_modeling-language-modeling.ipynb.
Converted 02c_modeling-question-answering.ipynb.
Converted 02d_modeling-token-classification.ipynb.
Converted 02e_modeling-text-generation.ipynb.
Converted index.ipynb.
